# Computational Astrophysics 2021
---
## Eduard Larrañaga

Observatorio Astronómico Nacional\
Facultad de Ciencias\
Universidad Nacional de Colombia

---

## Exercise. Linear Regression using `SciKit Learn`

### About this notebook

In this exercise, you will analize some data from the Bolshoi simulation to find linear correlations between some variables. 

---

## Regression with dark matter halos

In this tutorial, you will practice how to carry out a simple regression task. The first step is to download a public dark matter halo catalogue from the Bolshoi simulation. It can be found at

http://www.slac.stanford.edu/~behroozi/Bolshoi_Catalogs/  

The catalogue at z=0 is named 'hlist_1.00035.list.gz'. Once it is unziped, the size of the datafile is about 8Gb and contains millions of samples. In order to illustrate the analysis of the data, a reduced version of this file, with approximately 21000 samples, is given with this notebook. This file is called 'bolshoi01.list' 

Complete information about the Bolshoi simulation can be found in the paper

A. Klypin, S. Trujillo-Gomez, J. Primack. *Halos and galaxies in the standard cosmological model: results from the Bolshoi simulation.*
https://arxiv.org/abs/1002.3660

In [ ]:
path='' #Define an empty string to use in case of local working

In [ ]:
# Working with google colab needs to mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# we define the path to the files
path = '/content/drive/MyDrive/Colab Notebooks/CA2021/10. Regression II/exercises/'

In [ ]:
#Import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#Plot in cells
%matplotlib inline

### Loading the data

In order to load the data from the file, we will use the function `numpy.genfromtxt()` which is similar to the function `numpy.loadtxt()`, that we have used before, but gives some options that help when dealing with missing values. Complete information about this function is found at

https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html

We will load the first 20000 samples.

In [ ]:
#Load data from ascii file
data = np.genfromtxt(path+"bolshoi01.list",comments='#',max_rows=20000)

From the header of the datafile, you can identify the features included. Some of them are

| Column | Feature |
|---|:--|
|10| Mvir: Halo mass (Msun/h)|
|11| Rvir: Halo radius (kpc/h comoving)|
|12| Rs: Scale radius (kpc/h comoving)|
|13| Vrms: Velocity dispersion (km/s physical)|
|14| mmp?: whether the halo is the most massive progenitor or not|
|15| scale_of_last_MM: scale factor of the last major merger (Mass ratio > 0.3)|
|16| Vmax: Maxmimum circular velocity (km/s physical)|
|17:19| X / Y / Z: Halo position (Mpc/h comoving)|
|20:22| VX / VY / VZ: Halo velocity (km/s physical)|
|23:25| JX / JY / JZ: Halo angular momenta ((Msun/h) * (Mpc/h) * km/s (physical))|
|26| Spin: Halo spin parameter|
|27| Breadth_first_ID: breadth-first ordering of halos within a tree|
|28| Depth_first_ID: depth-first ordering of halos within a tree|
|29| Tree_root_ID: ID of the halo at the last timestep in the tree|
|30| Orig_halo_ID: Original halo ID from halo finder|
|31| Snap_num: Snapshot number from which halo originated|
|32| Next_coprogenitor_depthfirst_ID: Depthfirst ID of next coprogenitor|
|33| Last_progenitor_depthfirst_ID: Depthfirst ID of last progenitor|
|34| Last_mainleaf_depthfirst_ID: Depthfirst ID of last progenitor on main progenitor branch|
|35| Rs_Klypin: Scale radius determined using Vmax and Mvir (see Rockstar paper)|
|36| Mvir_all: Mass enclosed within the specified overdensity, including unbound particles (Msun/h)|
|37:40| M200b--M2500c: Mass enclosed within specified overdensities (Msun/h)|
|41| Xoff: Offset of density peak from average particle position (kpc/h comoving)|
|42| Voff: Offset of density peak from average particle velocity (km/s physical)|
|43| Spin_Bullock: Bullock spin parameter (J/(sqrt(2)*GMVR))|
|44:45| b_to_a, c_to_a: Ratio of second and third largest shape ellipsoid axes (B and C) to largest shape ellipsoid axis (A) (dimensionless).<br> Shapes are determined by the method in Allgood et al. (2006)|
|46:48| A[x],A[y],A[z]: Largest shape ellipsoid axis (kpc/h comoving)|
|49:50| b_to_a, c_to_a: Ratio of second and third largest shape ellipsoid axes (B and C) to largest shape ellipsoid axis (A) (dimensionless).<br> Shapes are determined by the method in Allgood et al. (2006). <br>(500c) indicates that only particles within R500c are considered|
|51:53| A[x],A[y],A[z]: Largest shape ellipsoid axis (kpc/h comoving).<br>(500c) indicates that only particles within R500c are considered|
|54| T/\|U\|: ratio of kinetic to potential energies|
|55:56| M_pe_*: Pseudo-evolution corrected masses (very experimental).Consistent Trees Version 1.0+. Includes fix for Rockstar spins & T/|U| (assuming T/|U| = column 53)|
|57|Macc: Mass at accretion|
|58|Mpeak: Peak mass over mass accretion history1|
|59|Vacc: Vmax at accretion|
|60|Vpeak: Peak Vmax over mass accretion history1|
|61|Halfmass_Scale: Scale factor at which the MMP reaches 0.5*Mpeak|
|62:64|Acc_Rate_\*: Halo mass accretion rates in Msun / h / yr <br>Inst: instantaneous; 100Myr: averaged over past 100Myr <br>X\*Tdyn: averaged over past X*virial dynamical time <br>Mpeak: Growth Rate of Mpeak, averaged from current z to z+0.5|

Now, we will extract some of these properties. (Since we are using machine learning, we will call the halo properties from now on 'features').

In [ ]:
#Define halo properties
#Each column gives us a different halo property. Here we define all those properties we want to use.
#If you feal like adding some more properties, try it out...
virial_mass   = np.log10(data[:,10])
virial_radius = np.log10(data[:,11])
concentration = np.log10(data[:,11] / data[:,12]) #Concentration is virial radius divided by scale length
velocity_disp = np.log10(data[:,13])
vmax          = np.log10(data[:,16])
spin          = np.log10(data[:,26])
b_to_a        = data[:,44]
c_to_a        = data[:,45]
energy_ratio  = data[:,54]
peak_mass     = np.log10(data[:,58])
peak_vmax     = np.log10(data[:,60])
halfmass_a    = data[:,61]
peakmass_a    = data[:,67]
acc_rate      = data[:,64]

You will then use the Pandas library to analyse this halo catalogue and to identify correlation between different halo properties. 

**1. Define a dataframe including the features describing the halos.**


In [ ]:
#Create Pandas Data Frame
#For easier handling we use the Pandas library.
#The following lines load all halo properties into a Pandas data frame called halos.
#Add all halo properties here:
halos = pd.DataFrame({
    'Virial Mass':virial_mass,
    'Virial Radius':virial_radius,
    'Concentration':concentration,
    'Vel. Dispersion':velocity_disp,
#Add the other quantities here!
})
halos

,Virial Mass,Virial Radius,Concentration,Vel. Dispersion
0,14.215638,3.054453,1.001863,2.984820
1,14.201124,3.049566,0.699512,2.947919
2,14.058426,3.001994,0.889922,2.907191
3,13.987175,2.978264,0.479754,2.879302
4,13.930796,2.959464,0.898464,2.872739
...,...,...,...,...
19995,10.283753,1.743768,1.431168,1.718585
19996,10.283753,1.743768,1.435631,1.678427
19997,10.283753,1.743768,1.133534,1.609914
19998,10.283753,1.743768,1.050305,1.632862


In [ ]:
halos.describe()

,Virial Mass,Virial Radius,Concentration,Vel. Dispersion
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10.781188,1.909596,1.144886,1.831052
std,0.486944,0.162315,0.317982,0.170236
min,10.283753,1.743768,0.002828,1.548021
25%,10.428944,1.792181,0.963105,1.707315
50%,10.632761,1.860134,1.126266,1.787248
75%,10.980821,1.976139,1.298915,1.907881
max,14.215638,3.054453,3.029893,2.984820


Note that there are 20000 complete samples (i.e. there are no missing values!). 

**2. In order to visualize the data, use the `dataframe.hist()` method to plot the histogram for each feature.**

In [ ]:
#Plot a histogram for each halo property
#Use the function .hist that every pandas data frame has
#20 bins should work well


This plot shows a histogram for each halo property. Some features, like the virial mass, the peak virial mass, and the virial radius start very high and then decline (they don't have a maximum). Some other features, like the half mass scale factor, the concentration, and the spin have a clear peak.

### Correlations in the data
We want to predict the halo concentration from the other features. It is helpful to first get an idea how to do this, by visualizing the datq, looking for correlations of each feature with the concentration. 


**3. Create the Scatter Matrix for all the features**

In [ ]:
attributes=['Feature 1', 'Feature 2', ...]
scatMatrix=pd.plotting.scatter_matrix(halos[attributes], figsize=(12,8))

**4. Use the `dataframe.corr()` method to obtain the correlation matrix of the dataframe.**

In [ ]:
#Compute correlation matrix and show highest correlations for a given property
#The .corr() function gives you the correlation matrix
[Insert CODE Here] 

It is also possible to obtain the correlation coefficients of a particular feature with the rest of them by using a subscript with the name of the feature,

In [ ]:
#You can then print the result with the ['Name of Feature'] subscript
#If you add .sort_values() the results will get sorted
halos.corr()['Name of the Feature']

**It looks that 'concentration' has the highest (anti-)'correlation' with [Insert Feature], although the 'correlation' with [Insert Feature] is also significant.**


**5 Create a scatter plot of the 'concentration' vs. the highest correlation feature.**

In [ ]:
#Show scatter plot for highest correlation
halos.plot(kind='scatter', x='Insert Feature', y='Concentration', alpha=0.1)

### Preparing the data for machine learning
In order to train the linear regressor, we first have to produce a set of arrays with the relevant features.

**6. Define the two arrays containing the 'concentration' and the highest correlation feature from the dataframe.** 

In [ ]:
#We cannot directly feed a Pandas data frame to scikit-learn.
#Therefore we need to transform our dataframe into a set of numpy arrays.


### Splitting the data into training and test sets

In the next step, you will use the Scikit-Learn library to predict the halo 'concentration' from the other halo properties using a simple linear regression. 

As is common in machine learning, we will want to test how well our solutions generalise to other data sets.
It doesn't help us, if the algorithm learned how to model a feature for our training set very well, if it completely fails when used with a different data set.  

Therefore, we need to split our data set into a training and a test set. 

**7. Split the dataset into a train and a test sets.**

In [ ]:
#Split train and test set randomly
#We use 20 per cent of the total data set as test set
#Use train_test_split to split the data
#Use random_state= with some integer to always have the same random numbers
X_train, X_test, y_train, y_test = ...

### Training a linear regressor
Now that we have prepared the data, we can simply select a regressor and fit the data.

**8. Train the linear model with the defined arrays**

In [ ]:
#Linear Regression
lin_reg = LinearRegression()
lin_reg.fit([INSERT HERE THE ARRAYS])

### Final evaluation on the test set
Now is time to test the linear model by computing the  prediction for the targets in the test set and meassure the final RMSE with respect to the knwon concentrations.

**9. Calcuate the RMSE using the predictions and the test set**

In [ ]:
#Compute Predictions and determine RMSE (=root mean squared error)
predictions = lin_reg.predict(...)
lin_mse = mean_squared_error(...)
lin_rmse = np.sqrt(...)
lin_rmse

So we get a RMSE of about [...]dex, i.e. about [...]%. For this little work, this is not too bad. Let's see how it looks like! For this we plot the predicted concentrations vs. the labels (i.e. the real concentrations).

**10. Plot the datapoints (train and test sets) together with the linear model** 

In [ ]:
#Plot prediction vs label
#plot predicted concentration vs. labels
#plot the 'perfect' result

### A Multidimensional Linear Regression

**11. Build a multilinear linear model to predict the 'concentration' from the two features with higher correlation coefficients.**